In [ ]:
# Create a UDP socket

import socket

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind the socket to an address and port
server_socket.bind(('localhost', 65433))
print("UDP Server is ready to receive messages...")

clients = set()  # Store client addresses

while True:
    # Receive data from client
    data, client_address = server_socket.recvfrom(2048)
    clients.add(client_address)
    print(f"Received from {client_address}: {data.decode()}")
    
    # Broadcast message to all connected clients
    for client in clients:
        if client != client_address:
            server_socket.sendto(data, client)

In [ ]:
# Create a UDP socket

import socket
import threading

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Define the server address
server_address = ('localhost', 65433)

# Function to receive messages
def receive_messages():
    while True:
        data, _ = client_socket.recvfrom(2048)
        print(f"\nReceived: {data.decode()}")

# Start a separate thread for receiving messages
threading.Thread(target=receive_messages, daemon=True).start()

# Send messages to the server
while True:
    message = input("You: ")
    if message.lower() == "exit":
        break
    client_socket.sendto(message.encode(), server_address)

client_socket.close()

In [ ]:
#  Exercise : 2
# Create a UDP socket
# Dictionary to store client IP addresses and ports

import socket

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind the socket to an address and port
server_socket.bind(('localhost', 65433))
print("UDP Server is ready to receive messages...")

clients = {}  # Dictionary to store client IP addresses and ports

while True:
    # Receive data from client
    data, client_address = server_socket.recvfrom(2048)
    
    # Store client address in the dictionary
    if client_address not in clients:
        clients[client_address] = client_address[0]  # Store only the IP
    
    print(f"Received from {client_address}: {data.decode()}")
    
    # Broadcast message to all connected clients
    for client in clients:
        if client != client_address:
            server_socket.sendto(data, client)

In [ ]:
# Exercise : 2
# Dictionary to store client IP addresses and ports
#  Create a UDP Client socket

import socket
import threading

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Define the server address
server_address = ('localhost', 65433)

# Function to receive messages
def receive_messages():
    while True:
        data, _ = client_socket.recvfrom(2048)
        print(f"\nReceived: {data.decode()}")

# Start a separate thread for receiving messages
threading.Thread(target=receive_messages, daemon=True).start()

# Send messages to the server
while True:
    message = input("You: ")
    if message.lower() == "exit":
        break
    client_socket.sendto(message.encode(), server_address)

client_socket.close()

In [ ]:
# Exercise 3: Add authentication step to the application in which the sever would
# ask the client for username and password to initiate the communication.

import socket

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind the socket to an address and port
server_socket.bind(('localhost', 65433))
print("UDP Server is ready to receive messages...")

clients = {}  # Dictionary to store client IP addresses and usernames
user_credentials = {"user1": "password1", "user2": "password2", "user3": "password3"}  # Predefined users

while True:
    # Receive authentication data from client
    auth_data, client_address = server_socket.recvfrom(2048)
    
    # If client is already authenticated, process their messages
    if client_address in clients:
        print(f"Received from {clients[client_address]} ({client_address}): {auth_data.decode()}")
        
        # Broadcast message to all connected clients
        for client in clients:
            if client != client_address:
                server_socket.sendto(auth_data, client)
        continue
    
    # Process authentication request
    try:
        username, password = auth_data.decode().split(',')
        if username in user_credentials and user_credentials[username] == password:
            server_socket.sendto(b"Authentication successful", client_address)
            clients[client_address] = username  # Store username linked to client address
            print(f"{username} authenticated from {client_address}")
        else:
            server_socket.sendto(b"Authentication failed", client_address)
    except ValueError:
        server_socket.sendto(b"Invalid authentication format", client_address)


In [ ]:
# client enter details for authentication
import socket
import threading

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Define the server address
server_address = ('localhost', 65433)

# Authenticate user
username = input("Enter username: ")
password = input("Enter password: ")
credentials = f"{username},{password}".encode()
client_socket.sendto(credentials, server_address)

# Receive authentication response
response, _ = client_socket.recvfrom(2048)
print(response.decode())
if response.decode() != "Authentication successful":
    client_socket.close()
    exit()

# Function to receive messages
def receive_messages():
    while True:
        data, _ = client_socket.recvfrom(2048)
        print(f"\nReceived: {data.decode()}")

# Start a separate thread for receiving messages
threading.Thread(target=receive_messages, daemon=True).start()

# Send messages to the server
while True:
    message = input("You: ")
    if message.lower() == "exit":
        break
    client_socket.sendto(message.encode(), server_address)

client_socket.close()

In [ ]:
# Exercise 4: ( ) Write functions that would encrypt the messages in the client
# and decrypt them in the server

In [ ]:
import socket
from cryptography.fernet import Fernet

# Generate and store encryption key (use the same key in the client)
key = Fernet.generate_key()
cipher_suite = Fernet(key)
print(f"Encryption Key (share with clients): {key.decode()}")

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind the socket to an address and port
server_socket.bind(('localhost', 65433))
print("UDP Server is ready to receive messages...")

clients = {}  # Dictionary to store client IP addresses and ports
user_credentials = {"user1": "password1", "user2": "password2"}  # Predefined users

while True:
    # Receive authentication data from client
    auth_data, client_address = server_socket.recvfrom(2048)
    username, password = auth_data.decode().split(',')
    
    if username in user_credentials and user_credentials[username] == password:
        server_socket.sendto(b"Authentication successful", client_address)
        clients[client_address] = username  # Store username linked to client address
        print(f"{username} authenticated from {client_address}")
    else:
        server_socket.sendto(b"Authentication failed", client_address)
        continue
    
    while True:
        encrypted_data, client_address = server_socket.recvfrom(2048)
        decrypted_message = cipher_suite.decrypt(encrypted_data).decode()
        print(f"Received from {clients[client_address]} ({client_address}): {decrypted_message}")
        
        # Broadcast encrypted message to all connected clients
        for client in clients:
            if client != client_address:
                server_socket.sendto(encrypted_data, client)

In [ ]:
import socket
import threading
from cryptography.fernet import Fernet

# Use the same key generated by the server
key = input("Enter the encryption key provided by the server: ").encode()
cipher_suite = Fernet(key)

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Define the server address
server_address = ('localhost', 65433)

# Authenticate user
username = input("Enter username: ")
password = input("Enter password: ")
credentials = f"{username},{password}".encode()
client_socket.sendto(credentials, server_address)

# Receive authentication response
response, _ = client_socket.recvfrom(2048)
print(response.decode())
if response.decode() != "Authentication successful":
    client_socket.close()
    exit()

# Function to receive messages
def receive_messages():
    while True:
        encrypted_data, _ = client_socket.recvfrom(2048)
        decrypted_message = cipher_suite.decrypt(encrypted_data).decode()
        print(f"\nReceived: {decrypted_message}")

# Start a separate thread for receiving messages
threading.Thread(target=receive_messages, daemon=True).start()

# Send messages to the server
while True:
    message = input("You: ")
    if message.lower() == "exit":
        break
    encrypted_message = cipher_suite.encrypt(message.encode())
    client_socket.sendto(encrypted_message, server_address)

client_socket.close()

In [ ]:
# Exercise 5: Check the API website and update the script to compare 
# the temperate between the university and the British Library.

In [ ]:
import socket
import requests

# Fetch weather data from a given location using Open Meteo API
def fetch_weather_data(latitude, longitude):
    api_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(api_url)
    if response.status_code == 200:
        weather_data = response.json()
        return weather_data["current_weather"]["temperature"]
    else:
        return None

# Compare the temperatures between Goldsmiths University and the British Library
def compare_temperatures():
    # Coordinates for Goldsmiths University, London
    university_lat = 51.4761
    university_lon = -0.0035

    # Coordinates for the British Library
    british_library_lat = 51.5291
    british_library_lon = -0.1270

    # Fetch temperatures for both locations
    university_temp = fetch_weather_data(university_lat, university_lon)
    british_library_temp = fetch_weather_data(british_library_lat, british_library_lon)

    if university_temp is not None and british_library_temp is not None:
        if university_temp > british_library_temp:
            message = f"The temperature at Goldsmiths University is higher than the British Library: {university_temp}°C vs {british_library_temp}°C."
        elif university_temp < british_library_temp:
            message = f"The temperature at the British Library is higher than Goldsmiths University: {british_library_temp}°C vs {university_temp}°C."
        else:
            message = f"The temperature is the same at both locations: {university_temp}°C."
    else:
        message = "Failed to fetch weather data for one or both locations."

    return message

# Send the weather comparison result using UDP
def send_weather_data(message):
    # Create UDP socket and send the message
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = ('localhost', 65433)
    client_socket.sendto(message.encode(), server_address)
    print("Weather comparison sent!")
    client_socket.close()

# Main function to run the application
def main():
    message = compare_temperatures()
    send_weather_data(message)

if __name__ == "__main__":
    main()


In [ ]:
import socket

# Set up UDP server to receive the weather comparison message
def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind(('localhost', 65433))
    print("Server is listening for messages...")

    while True:
        message, address = server_socket.recvfrom(1024)  # Buffer size
        print(f"Received message: {message.decode()}")

if __name__ == "__main__":
    start_server()
